# Show Me the City Data
> A tutorial for programatically reading Open Data from a Socrata database.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: ../images/cincy_skyline.jpeg

## In this Post

Let's bring some first post energy!  I am excited to kick off this blog with a post about accessing Open Data in an easy, programmatic manner.

This notebook will demonstrate how to programatically read in Open Data from Socrata databases.  Socrata is a company commissioned by a large number of cities to host open data platforms.

These Open Data platforms are searchable and most importantly the data can be queried via an API.  These queries can be quite simple, or more complex.  We will dive into examples of both and be querying data in no time!

## Why is this Important

Querying and accessing data is a fundamental step of any data science workflow.  The mechanics of accessing data can get very messy, especially when it comes to government data.  Examples of this messiness include manually downloading files and saving to a user-defined location, appending multiple Excel files, scraping PDFs, and the list goes on.  Bottom line: it can get MESSY.

Luckily, the infrastructure provided by Socrata makes for a seamless experience that can be replicated by anyone with the Internet and Python.  You read that correctly, everything demonstrated in this post can be replicated with very little setup.  This is a huge perk for the sake of collaboration.  Additionally, in the event that an analysis needs to be re-run, perhaps on more recent data, the ability to query data programatically allows for minimal rework and minimal room for error.  Alright, enough hype let's get to it!

## Setup

Before we get rolling, it is important to be working in an environment with the necessary packages installed and available.  I like using Conda to manage environments (this sounds like a future blog post) and for the demonstration below it will be important to have pandas installed in your environment.

In [2]:
# load pandas
import pandas as pd

## Finding an API Endpoint

Before we dive into actually reading data, it is important to know where to look for potential datasets.  Most cities have their Open Data tied to the city's website.  In this case we are going to explore the City of Cincinnati's Open Data Portal, which can be found at this [link](https://data.cincinnati-oh.gov).

![](../images/2021-08-19-open-data-portal.png)


At this point, we can navigate into one of the categories or search for a specific dataset.  In this case, we select the 'Safety' category and will then choose the Police Calls dataset.

![](../images/2021-08-19-safety-data.png)


We are now able to Copy the API Endpoint by selecting the 'API' button and the 'Copy' button next to the endpoint.

![](../images/2021-08-19-PDI-endpoint.png)




## Reading in Data via API

You might notice that the API endpoint is just a link - try accessing the link below in your browser.  You will notice that the result is just a JSON string in your browser - this is the data we are pulling in imminently!

In order to read this into a pandas DataFrame, we utilize the pandas `read_json` function.  This function only has one required argument, `path_or_buf` and we set the value to our API endpoint.  More information on `read_json` can be found by looking at its docstring (simply run `?read_json`).

Note that in this initial read, we only pull back 1,000 total rows.  This is the default response when no other parameters are added to the endpoint URL.

In [159]:
# define a variable to store our endpoint URL to be used throughout the post
crime_data_endpoint = 'https://data.cincinnati-oh.gov/resource/k59e-2pvf.json'

pd.read_json(path_or_buf = crime_data_endpoint)

,instanceid,incident_no,date_reported,date_from,date_to,clsd,ucr,dst,beat,offense,...,suspect_age,totalnumbervictims,ucr_group,community_council_neighborhood,zip,suspect_race,suspect_ethnicity,suspect_gender,totalsuspects,theft_code
0,75092BE1-203C-4251-A197-6A4CD492E803,129023054,2012-08-15T19:40:00.000,2012-08-15T01:00:00.000,2012-08-15T09:00:00.000,Z--EARLY CLOSED,551,2,1.0,BURGLARY,...,UNKNOWN,1.0,BURGLARY/BREAKING ENTERING,N/A,45206,NaN,NaN,NaN,NaN,NaN
1,D229DDF6-0F66-4AC5-AD13-7F83D2566A91,179006481,2017-03-02T17:16:00.000,2017-03-02T17:10:00.000,2017-03-02T17:15:00.000,J--CLOSED,810,3,2.0,ASSAULT,...,31-40,1.0,PART 2 MINOR,N/A,45205,WHITE,NOT OF HISPANIC ORIG,MALE,1.0,NaN
2,961D1BFE-C1BB-4B44-A8F9-2599F7C83249,179020029,2017-06-01T10:35:00.000,2017-06-01T10:25:00.000,2017-06-01T10:30:00.000,J--CLOSED,600,5,2.0,THEFT,...,UNKNOWN,1.0,THEFT,N/A,45225,NaN,NaN,NaN,NaN,24O-MOTOR VEHICLE THEFT
3,847165DB-0557-4E36-8692-900D6C456055,129005579,2012-03-04T03:11:00.000,2012-03-04T03:10:00.000,2012-03-04T03:11:00.000,F--CLEARED BY ARREST - ADULT,301,5,1.0,ROBBERY,...,UNKNOWN,1.0,ROBBERY,N/A,45219,UNKNOWN,UNKNOWN,MALE,1.0,NaN
4,C1292536-04EB-42BC-9B80-263D731ABD0E,179053792,2017-12-17T09:53:00.000,2017-12-10T18:00:00.000,2017-12-11T08:00:00.000,Z--EARLY CLOSED,600,5,2.0,THEFT,...,UNKNOWN,2.0,THEFT,N/A,45225,NaN,NaN,NaN,NaN,23F-THEFT FROM MOTOR VEHICLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,67038B33-6C6F-4720-9C17-706C841B6662,159006129,2015-03-16T10:55:00.000,2015-03-13T16:00:00.000,2015-03-16T10:30:00.000,Z--EARLY CLOSED,552,3,4.0,BREAKING AND ENTERING,...,UNKNOWN,1.0,BURGLARY/BREAKING ENTERING,N/A,45214,NaN,NaN,NaN,NaN,NaN
996,7D66BB58-BCE4-4A2B-B0FA-6405696E9A11,11104733,2011-11-23T23:05:00.000,2011-11-23T18:30:00.000,2011-11-23T23:00:00.000,Z--EARLY CLOSED,600,1,1.0,THEFT,...,UNKNOWN,1.0,THEFT,N/A,45202,NaN,NaN,NaN,NaN,23F-THEFT FROM MOTOR VEHICLE
997,AEA518CF-A343-49D6-9F47-27A9225D00DC,129014979,2012-06-01T01:45:00.000,2012-05-31T22:30:00.000,2012-06-01T01:00:00.000,J--CLOSED,1493,4,5.0,CRIMINAL DAMAGING/ENDANGERING,...,UNKNOWN,2.0,PART 2 MINOR,N/A,45237,NaN,NaN,NaN,NaN,NaN
998,3C1B16A9-8A1F-47B8-AB30-505670F60DF3,179029164,2017-07-21T11:45:00.000,2017-07-20T18:30:00.000,2017-07-20T18:30:00.000,Z--EARLY CLOSED,600,3,3.0,THEFT,...,UNKNOWN,1.0,THEFT,N/A,45238,NaN,NaN,NaN,NaN,23H-ALL OTHER LARCENY


## Socrata Query Language (SoQL)

In the above example, we simply query the entire dataset (albeit a sample of 1,000 rows) given we did not make any tweaks to the base URL.  However, we are able to arrive at fairly robust queries using the SoQL querying language. SoQL is a querying language developed by Socrata in order to make the process of requesting data via their API more efficient.  In their [documentation](https://dev.socrata.com/docs/queries/) they mention their intentions of mirroring the spirit of SQL and I believe they accomplished that goal quite well.

Being well versed in this querying language provides enormous benefits in the case of processing time.  It is far more efficient to execute data processing against the Socrata servers prior to bringing in data to your local machine.  For the remainder of this post we will explore examples of utilizing SoQL queries to refine our data before pulling it into our local machines.

### Example 1: Simple Filter

In our first example, we are going to execute a simple filter on our data by updating the `$WHERE` parameter.  As you would imagine, the default value for the `$WHERE` parameter is No Filter.

In our initial query above, we see that there is an `offense` column with a number of different values, stored as strings.  For this exercise, we are going to filter to instances where `offense` is equal to 'THEFT'.

In practice, we are able to execute this by modifying our API endpoint with the simple filter clause stamped on at the end.  To signal that we are going to add a SoQL clause to our endpoint, we first add a question mark (?) at the end of the initial endpoint, immediately following '.json':  `https://data.cincinnati-oh.gov/resource/k59e-2pvf.json?`

Now, we must add the actual where clause.  In this case, we first declare that we are updating the `$WHERE` parameter by adding `where=`.  We now add the filter statement, in this case `offense = 'THEFT'`.  This gives us a final query that looks like the following: 

`https://data.cincinnati-oh.gov/resource/k59e-2pvf.json?$WHERE=offense='THEFT'`.

> Note: from this point forward we will simply reference `crime_data_endpoint` when generating queries.  We will take advantage of f-String formatting that allows us to pass through a variable to a string statement (i.e. if `name` is a variable defined in the environment as the value of `'Joe'`, the statement `f"I am {name}"` will evaluate to `'I am Joe'`.

In [160]:
#collapse-output
pd.read_json(f"{crime_data_endpoint}?$WHERE=offense='THEFT'")

,instanceid,incident_no,date_reported,date_from,date_to,clsd,ucr,dst,beat,offense,...,suspect_age,totalnumbervictims,ucr_group,community_council_neighborhood,zip,suspect_race,suspect_ethnicity,suspect_gender,totalsuspects,side
0,961D1BFE-C1BB-4B44-A8F9-2599F7C83249,179020029,2017-06-01T10:35:00.000,2017-06-01T10:25:00.000,2017-06-01T10:30:00.000,J--CLOSED,600,5,2.0,THEFT,...,UNKNOWN,1.0,THEFT,N/A,45225,NaN,NaN,NaN,NaN,NaN
1,C1292536-04EB-42BC-9B80-263D731ABD0E,179053792,2017-12-17T09:53:00.000,2017-12-10T18:00:00.000,2017-12-11T08:00:00.000,Z--EARLY CLOSED,600,5,2.0,THEFT,...,UNKNOWN,2.0,THEFT,N/A,45225,NaN,NaN,NaN,NaN,NaN
2,853035AE-12A4-4F27-8960-78DB3B0D8B6E,139024773,2013-09-11T21:30:00.000,2013-09-11T18:45:00.000,2013-09-11T19:41:00.000,Z--EARLY CLOSED,600,2,NaN,THEFT,...,UNKNOWN,1.0,THEFT,N/A,45226,NaN,NaN,NaN,NaN,NaN
3,17A8E469-B955-4C9E-A39A-09A8610FB180,179050509,2017-11-27T14:35:00.000,2017-11-23T16:00:00.000,2017-11-23T16:00:00.000,F--CLEARED BY ARREST - ADULT,600,2,4.0,THEFT,...,18-25,1.0,THEFT,N/A,45230,WHITE,HISPANIC ORIGIN,MALE,1.0,NaN
4,FFBEE182-CE3D-4E34-9A95-524C07AC1695,11005036,2010-10-26T10:48:00.000,2010-10-23T02:00:00.000,2010-10-23T02:15:00.000,J--CLOSED,600,1,3.0,THEFT,...,UNKNOWN,1.0,THEFT,N/A,45203,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1951F631-6E0B-495C-A071-189333C1D1CF,169015730,2016-06-28T06:45:00.000,2016-06-28T01:00:00.000,2016-06-28T06:30:00.000,J--CLOSED,600,5,2.0,THEFT,...,UNKNOWN,1.0,THEFT,N/A,45225,NaN,NaN,NaN,NaN,NaN
996,B922AC23-0156-4C8B-AF72-298C69BED2C2,129016496,2012-06-18T07:40:00.000,2012-06-14T17:30:00.000,2012-06-18T06:30:00.000,Z--EARLY CLOSED,600,1,3.0,THEFT,...,UNKNOWN,1.0,THEFT,N/A,45203,NaN,NaN,NaN,NaN,NaN
997,83F56B29-D15A-407E-9321-5841397CE935,129031483,2012-10-27T22:30:00.000,2012-10-27T07:00:00.000,2012-10-27T20:00:00.000,Z--EARLY CLOSED,600,2,2.0,THEFT,...,UNKNOWN,1.0,THEFT,N/A,45209,NaN,NaN,NaN,NaN,NaN
998,59B7E1FB-947D-4BC2-B42D-6F6E59FD039D,159018103,2015-07-13T08:52:00.000,2015-07-11T16:00:00.000,2015-07-13T10:50:00.000,Z--EARLY CLOSED,600,CENTRAL BUSINESS,1.0,THEFT,...,UNKNOWN,1.0,THEFT,N/A,45202,NaN,NaN,NaN,NaN,NaN


With the query above, we see the data returned is exclusively limited to 'THEFT' offenses as desired.  We are now going to demonstrate the ability to string multiple `WHERE` clauses into the same query.

In order to do so, we tap into the logical operators allowed within the SoQL framework.  The following operators are available as illustrated in the [documentation](https://dev.socrata.com/docs/queries/where.html):
- `AND` (&)
- `OR` (|)
- `NOT`
- `IS NULL`
- `IS NOT NULL`

In this case, we are going to add the condition that an incident must have occurred in the `45202` zip code.  To accomplish this, we simply add the `&` symbol and an additional condition of `zip=45202`.

> Note: the SoQL documentation shows examples that include spaces in the endpoint URL.  The `pd.read_json()` function does not accept URLs with spaces.  Do your best to avoid using spaces, or replace the space with its URL encoding (in the case of a space that is `%20`).

In [161]:
#collapse-output
pd.read_json(f"{crime_data_endpoint}?$WHERE=offense='THEFT'&zip=45202")

,instanceid,incident_no,date_reported,date_from,date_to,clsd,ucr,dst,beat,offense,...,suspect_age,totalnumbervictims,ucr_group,community_council_neighborhood,zip,suspect_race,suspect_ethnicity,suspect_gender,totalsuspects,side
0,5AAC40C8-755D-4163-9DFF-5D51D5E350B2,129017582,2012-06-18T12:55:00.000,2012-06-18T12:00:00.000,2012-06-18T12:45:00.000,F--CLEARED BY ARREST - ADULT,600,4,2.0,THEFT,...,UNKNOWN,1,THEFT,N/A,45202,NaN,NaN,NaN,NaN,NaN
1,87EC8A12-B8C1-49D7-B77F-613741A60755,179050214,2017-11-25T08:40:00.000,2017-11-25T01:00:00.000,2017-11-25T01:30:00.000,Z--EARLY CLOSED,600,CENTRAL BUSINESS,1.0,THEFT,...,UNKNOWN,1,THEFT,N/A,45202,NaN,NaN,NaN,NaN,NaN
2,4FBD01D3-FE0A-4B66-9798-930351A11081,129028052,2012-09-26T22:10:00.000,2012-09-26T21:30:00.000,2012-09-26T22:00:00.000,Z--EARLY CLOSED,600,1,2.0,THEFT,...,UNKNOWN,1,THEFT,N/A,45202,NaN,NaN,NaN,NaN,NaN
3,F36D6971-0AD4-45E3-93E7-44A4C1C220EF,179C000285,2017-06-02T09:22:24.000,2017-05-18T11:00:00.000,2017-05-18T14:00:00.000,Z--EARLY CLOSED,600,CENTRAL BUSINESS,1.0,THEFT,...,UNKNOWN,1,THEFT,N/A,45202,NaN,NaN,NaN,NaN,NaN
4,6B3CB5AB-56F8-44A8-BB49-49F1D87367CD,189026973,2018-10-06T13:05:00.000,2018-10-06T13:00:00.000,2018-10-06T12:00:00.000,J--CLOSED,600,CENTRAL BUSINESS,2.0,THEFT,...,UNKNOWN,1,THEFT,N/A,45202,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,496C05FC-A9F8-44E3-A236-8DC4C645C649,159C000036,2015-01-22T01:08:41.000,2015-01-21T22:00:00.000,2015-01-22T00:55:00.000,Z--EARLY CLOSED,600,1,2.0,THEFT,...,UNKNOWN,1,THEFT,N/A,45202,NaN,NaN,NaN,NaN,NaN
996,CEB4AA17-8C35-4D4B-89A4-47805B8E5EB1,139023763,2013-08-29T03:53:00.000,2013-08-29T03:40:00.000,2013-08-29T03:53:00.000,F--CLEARED BY ARREST - ADULT,600,1,1.0,THEFT,...,UNKNOWN,4,THEFT,N/A,45202,NaN,NaN,NaN,NaN,NaN
997,D270C30F-981E-48EF-9597-78B709A03638,11105023,2011-12-19T12:15:00.000,2011-12-16T12:00:00.000,2011-12-16T12:15:00.000,J--CLOSED,600,1,1.0,THEFT,...,UNKNOWN,1,THEFT,N/A,45202,NaN,NaN,NaN,NaN,NaN
998,C0734EA5-6307-4DFF-86AF-1906D0A89C5B,139016219,2013-06-25T15:45:00.000,2013-06-23T19:00:00.000,2013-06-23T19:05:00.000,Z--EARLY CLOSED,600,1,2.0,THEFT,...,UNKNOWN,1,THEFT,N/A,45202,NaN,NaN,NaN,NaN,NaN


### Example 2: Date Filter

Working with dates can be finnicky in any scenario.  In this example, we aim to illustrate how to introduce a simple filter to limit the date range of our returned crime instances.

As you can see and infer from above, we have several fields that are represented as timestamps.  For this exercise, we are going to specifically focus on the `date_reported` field.  Let's explore how we would only look at records for which the `date_reported` value occurred within the last year.

For this exercise we will take advantage of the `datetime` library that assists with time and date operations.

In [162]:
# import datetime library
import datetime as dt

# dynamically generate today's date
today = dt.date.today()

# dynamically arrive at the date exactly 365 days ago
year_ago_today = today - dt.timedelta(days = 365)

print(year_ago_today)

2020-08-23


Now that we have our date for exactly one year ago, we can utilize it within our API call.  Similar to the simple filters above, we simply create a `$WHERE` clause, but in this case we look at instances where the `date_reported` field satisfies the condition of being after our `year_ago_today` value.

> Note: it is important to enclose the value for `year_ago_today` in quotes.  In this case I use single quotes for within the query, and enclose the entire query using double quotes.

In [163]:
#collapse-output
pd.read_json(f"{crime_data_endpoint}?$WHERE=date_reported>='{year_ago_today}'")

,instanceid,incident_no,date_reported,date_from,date_to,clsd,ucr,dst,beat,offense,...,zip,longitude_x,latitude_x,floor,side,opening,suspect_race,suspect_ethnicity,suspect_gender,totalsuspects
0,13624621-3CB9-49B5-8106-E99B589D3EE3,209018990,2020-08-23T00:00:00.000,2020-08-22T19:00:00.000,2020-08-23T00:00:00.000,Z--EARLY CLOSED,600.0,1,3,THEFT,...,45202.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,08DCA723-F1D9-4A9F-AD89-30A6993F7668,209018825,2020-08-23T00:33:00.000,2020-08-23T00:33:00.000,2020-08-23T00:38:00.000,F--CLEARED BY ARREST - ADULT,810.0,4,2,ASSAULT,...,45206.0,-84.487036,39.136701,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9BDDA414-5B4D-4301-B84D-234496CDE330,209018850,2020-08-23T00:36:00.000,2020-08-22T23:00:00.000,2020-08-23T00:30:00.000,F--CLEARED BY ARREST - ADULT,862.0,CENTRAL BUSINESS,1,DOMESTIC VIOLENCE,...,45202.0,-84.513128,39.099431,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,84DFCE4D-0FEB-4D4B-9AD9-90521B34211C,209018827,2020-08-23T01:07:00.000,2020-08-23T01:00:00.000,2020-08-23T01:07:00.000,K--UNFOUNDED,600.0,5,4,THEFT,...,45224.0,-84.545400,39.202558,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3DD34BF6-75FC-4536-8F6C-906232FB33BE,209018829,2020-08-23T01:50:00.000,2020-08-23T01:30:00.000,2020-08-23T01:45:00.000,F--CLEARED BY ARREST - ADULT,862.0,3,2,DOMESTIC VIOLENCE,...,45205.0,-84.572002,39.122697,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0F0D26CB-0428-43DC-8AC6-A60016234A18,209019851,2020-09-02T10:51:00.000,2020-09-02T00:15:00.000,2020-09-02T01:00:00.000,Z--EARLY CLOSED,600.0,4,5,THEFT,...,45237.0,-84.459236,39.204974,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,A3FA9007-5870-411C-8430-43B5C8449A2F,209019870,2020-09-02T11:34:00.000,2020-09-01T23:00:00.000,2020-09-02T11:30:00.000,J--CLOSED,600.0,2,1,THEFT,...,45207.0,-84.473283,39.141627,NaN,6 - UNKNOWN,NaN,NaN,NaN,NaN,NaN
997,99A181EB-06B1-4676-B5EB-6B98F24E0B0E,209C000687,2020-09-02T11:36:52.000,2020-08-31T19:00:00.000,2020-09-01T08:00:00.000,Z--EARLY CLOSED,600.0,CENTRAL BUSINESS,1,THEFT,...,45202.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,7E910A4D-A00E-4092-8A7D-56DF7AC9B144,209019858,2020-09-02T11:45:00.000,2020-09-01T20:30:20.000,2020-09-02T10:30:00.000,J--CLOSED,600.0,1,3,THEFT,...,45203.0,-84.538450,39.105948,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Example 3: Toggling Row Limit

By default, the row limit returned by the Socrata API is 1,000.  This is great for datasets smaller than 1,000 rows and illustrative blog posts, but for anything that actually matters it will be woefully insufficient.  Hence, we will now walk through the means for adjusting the row limit and integrating this into our queries.

This mechanic is really quite straightforward.  Similar to other SoQL query arguments, we simply define the parameter we are adjusting by adding `$LIMIT`, and then specifying the value for which to limit the row counts.  In the example below we limit this to `10`, however, you could ramp it up as large or small as needed.

> Note: I have not found an 'unlimited' value for this argument, thus if I know I would like to return all records I simply pass through an uncharacteristically large value that would not be anywhere near the max size of the data.  This is not a perfect solution but gets the job done when you know the rough size of your data.

In [164]:
#collapse-output
pd.read_json(f"{crime_data_endpoint}?$LIMIT=10")

,instanceid,incident_no,date_reported,date_from,date_to,clsd,ucr,dst,beat,offense,...,suspect_age,totalnumbervictims,ucr_group,community_council_neighborhood,zip,suspect_race,suspect_ethnicity,suspect_gender,totalsuspects,theft_code
0,75092BE1-203C-4251-A197-6A4CD492E803,129023054,2012-08-15T19:40:00.000,2012-08-15T01:00:00.000,2012-08-15T09:00:00.000,Z--EARLY CLOSED,551,2,1.0,BURGLARY,...,UNKNOWN,1,BURGLARY/BREAKING ENTERING,N/A,45206,NaN,NaN,NaN,NaN,NaN
1,D229DDF6-0F66-4AC5-AD13-7F83D2566A91,179006481,2017-03-02T17:16:00.000,2017-03-02T17:10:00.000,2017-03-02T17:15:00.000,J--CLOSED,810,3,2.0,ASSAULT,...,31-40,1,PART 2 MINOR,N/A,45205,WHITE,NOT OF HISPANIC ORIG,MALE,1.0,NaN
2,961D1BFE-C1BB-4B44-A8F9-2599F7C83249,179020029,2017-06-01T10:35:00.000,2017-06-01T10:25:00.000,2017-06-01T10:30:00.000,J--CLOSED,600,5,2.0,THEFT,...,UNKNOWN,1,THEFT,N/A,45225,NaN,NaN,NaN,NaN,24O-MOTOR VEHICLE THEFT
3,847165DB-0557-4E36-8692-900D6C456055,129005579,2012-03-04T03:11:00.000,2012-03-04T03:10:00.000,2012-03-04T03:11:00.000,F--CLEARED BY ARREST - ADULT,301,5,1.0,ROBBERY,...,UNKNOWN,1,ROBBERY,N/A,45219,UNKNOWN,UNKNOWN,MALE,1.0,NaN
4,C1292536-04EB-42BC-9B80-263D731ABD0E,179053792,2017-12-17T09:53:00.000,2017-12-10T18:00:00.000,2017-12-11T08:00:00.000,Z--EARLY CLOSED,600,5,2.0,THEFT,...,UNKNOWN,2,THEFT,N/A,45225,NaN,NaN,NaN,NaN,23F-THEFT FROM MOTOR VEHICLE
5,853035AE-12A4-4F27-8960-78DB3B0D8B6E,139024773,2013-09-11T21:30:00.000,2013-09-11T18:45:00.000,2013-09-11T19:41:00.000,Z--EARLY CLOSED,600,2,NaN,THEFT,...,UNKNOWN,1,THEFT,N/A,45226,NaN,NaN,NaN,NaN,23F-THEFT FROM MOTOR VEHICLE
6,E8A3F779-6839-43D5-81C4-1A07D685C951,199003757,2019-02-22T21:36:00.000,2019-02-22T21:30:00.000,2019-02-22T21:40:00.000,Z--EARLY CLOSED,810,4,1.0,ASSAULT,...,UNKNOWN,2,PART 2 MINOR,N/A,45219,BLACK,UNKNOWN,MALE,2.0,NaN
7,17A8E469-B955-4C9E-A39A-09A8610FB180,179050509,2017-11-27T14:35:00.000,2017-11-23T16:00:00.000,2017-11-23T16:00:00.000,F--CLEARED BY ARREST - ADULT,600,2,4.0,THEFT,...,18-25,1,THEFT,N/A,45230,WHITE,HISPANIC ORIGIN,MALE,1.0,23D-THEFT FROM BUILDING
8,D7B24514-318A-4D0D-9B00-54E369EF0356,129014121,2012-05-28T21:00:00.000,2012-05-28T20:38:00.000,2012-05-28T20:40:00.000,J--CLOSED,1493,1,2.0,CRIMINAL DAMAGING/ENDANGERING,...,UNKNOWN,1,PART 2 MINOR,N/A,45202,NaN,NaN,NaN,NaN,NaN
9,6F3F5D41-24A9-490F-84A0-0430314A341F,51007218,2010-11-26T10:35:00.000,2010-11-26T00:30:00.000,2010-11-26T00:35:00.000,Z--EARLY CLOSED,1493,5,1.0,CRIMINAL DAMAGING/ENDANGERING,...,UNKNOWN,1,PART 2 MINOR,N/A,45220,NaN,NaN,NaN,NaN,NaN


In [165]:
#collapse-output
pd.read_json(f"{crime_data_endpoint}?$WHERE=offense='THEFT'&$LIMIT=5")

,instanceid,incident_no,date_reported,date_from,date_to,clsd,ucr,dst,beat,offense,...,victim_gender,suspect_age,totalnumbervictims,ucr_group,community_council_neighborhood,zip,suspect_race,suspect_ethnicity,suspect_gender,totalsuspects
0,961D1BFE-C1BB-4B44-A8F9-2599F7C83249,179020029,2017-06-01T10:35:00.000,2017-06-01T10:25:00.000,2017-06-01T10:30:00.000,J--CLOSED,600,5,2.0,THEFT,...,MALE,UNKNOWN,1,THEFT,N/A,45225,NaN,NaN,NaN,NaN
1,C1292536-04EB-42BC-9B80-263D731ABD0E,179053792,2017-12-17T09:53:00.000,2017-12-10T18:00:00.000,2017-12-11T08:00:00.000,Z--EARLY CLOSED,600,5,2.0,THEFT,...,MALE,UNKNOWN,2,THEFT,N/A,45225,NaN,NaN,NaN,NaN
2,853035AE-12A4-4F27-8960-78DB3B0D8B6E,139024773,2013-09-11T21:30:00.000,2013-09-11T18:45:00.000,2013-09-11T19:41:00.000,Z--EARLY CLOSED,600,2,NaN,THEFT,...,MALE,UNKNOWN,1,THEFT,N/A,45226,NaN,NaN,NaN,NaN
3,17A8E469-B955-4C9E-A39A-09A8610FB180,179050509,2017-11-27T14:35:00.000,2017-11-23T16:00:00.000,2017-11-23T16:00:00.000,F--CLEARED BY ARREST - ADULT,600,2,4.0,THEFT,...,FEMALE,18-25,1,THEFT,N/A,45230,WHITE,HISPANIC ORIGIN,MALE,1.0
4,FFBEE182-CE3D-4E34-9A95-524C07AC1695,11005036,2010-10-26T10:48:00.000,2010-10-23T02:00:00.000,2010-10-23T02:15:00.000,J--CLOSED,600,1,3.0,THEFT,...,FEMALE,UNKNOWN,1,THEFT,N/A,45203,NaN,NaN,NaN,NaN


### Example 4: Select Subset of Columns

In this example we walkthrough the method for selecting only a subset of the columns in a dataframe.  This improves performance when you only need to work with a subset of columns by reducing the size of the dataframe.

The syntax is fairly straightforward and mirrors the methods for adding other parameters to our query.  Simply add a `$SELECT=` clause to the URL and list columns to include, separated by commas.

> Note: again, it is important to avoid spaces within the URL.  There is functionality to select columns under new names, like so: `$SELECT=suspect_gender AS gender`.  However, in this case we need to clean up the query to avoid any spaces in the URL by replacing them with the `%20` notation: `$SELECT=suspect_gender%20AS%20gender`.

In [166]:
#collapse-output
pd.read_json(f"{crime_data_endpoint}?$SELECT=incident_no,date_reported,suspect_gender")

,incident_no,date_reported,suspect_gender
0,129023054,2012-08-15T19:40:00.000,NaN
1,179006481,2017-03-02T17:16:00.000,MALE
2,179020029,2017-06-01T10:35:00.000,NaN
3,129005579,2012-03-04T03:11:00.000,MALE
4,179053792,2017-12-17T09:53:00.000,NaN
...,...,...,...
995,209015661,2020-07-20T04:00:00.000,MALE
996,129008180,2012-03-24T22:47:00.000,NaN
997,149015152,2014-06-25T11:31:00.000,MALE
998,179025466,2017-07-05T11:45:00.000,MALE


### Example 5: Putting it All Together with `$query`

Up until this point, we have walked through the parameters that can be modified within a query on an individual basis.  It was noted that multiple parameters can be combined by adding an `&` symbol between parameters, leading to more complex queries.  However, there is an easier way to string these parameters together, and it manifests itself within the `$query` parameter.

In these examples we will demonstrate how to string together a full-fledged query using the single `$query` parameter.  These queries will strongly resemble standard SQL queries that lack a `FROM` clause.  

To make things simpler, I have created a function (`generate_query`) to take care of some of the less readable pieces of these queries, notably the requirement to replace spaces (`' '`) with `'%20'`.  Furthermore, this function breaks the components of the query into chunks, making it very human readable.

In [167]:
# function takes five arguments (endpoint_url and 4 components of the query) 
# function returns a full, cleaned up API call
def generate_query(endpoint_url, select_clause, where_clause, group_clause, order_by_clause):

    raw_query = (f"{endpoint_url}?$query="
                 f"{select_clause}%20"
                 f"{where_clause}%20"
                 f"{group_clause}%20"
                 f"{order_by_clause}"
                )
    
    final_query = str.replace(raw_query, " ", "%20")
    
    return final_query

Now, with our function in hand let's accomplish the following within two separate queries:
1. Select the columns `date_reported` and `offense` for records where `suspect_gender` is 'MALE'
2. Count the number of records by offense type for all crimes within the past year, order by the count in descending order

In [168]:
## 1. Select the columns `date_reported` and `offense` for records where `suspect_gender` is 'MALE'

first_query = generate_query(endpoint_url = crime_data_endpoint,
               select_clause = "select date_reported, offense",
               where_clause = "where offense = 'THEFT'",
               group_clause = "",
               order_by_clause = "")

print(first_query)

https://data.cincinnati-oh.gov/resource/k59e-2pvf.json?$query=select%20date_reported,%20offense%20where%20offense%20=%20'THEFT'%20%20


In [169]:
#collapse-output
pd.read_json(first_query)

,date_reported,offense
0,2017-06-01T10:35:00.000,THEFT
1,2017-12-17T09:53:00.000,THEFT
2,2013-09-11T21:30:00.000,THEFT
3,2017-11-27T14:35:00.000,THEFT
4,2010-10-26T10:48:00.000,THEFT
...,...,...
995,2010-07-12T03:40:00.000,THEFT
996,2015-06-01T10:30:00.000,THEFT
997,2016-06-02T15:43:00.000,THEFT
998,2011-08-24T14:24:00.000,THEFT


In [170]:
## 2. Count the number of records by offense type for all crimes within the past year, 
##    order by the count in descending order

second_query = generate_query(endpoint_url = crime_data_endpoint,
                              select_clause = "select offense, count(*) as n",
                              where_clause = f"where date_reported>='{year_ago_today}'",
                              group_clause = "group by offense",
                              order_by_clause = "order by count(*) desc"
                             )

print(second_query)

https://data.cincinnati-oh.gov/resource/k59e-2pvf.json?$query=select%20offense,%20count(*)%20as%20n%20where%20date_reported>='2020-08-23'%20group%20by%20offense%20order%20by%20count(*)%20desc


In [171]:
#collapse-output
pd.read_json(second_query)

,offense,n
0,THEFT,9688
1,CRIMINAL DAMAGING/ENDANGERING,4319
2,ASSAULT,3064
3,DOMESTIC VIOLENCE,1808
4,FELONIOUS ASSAULT,1388
...,...,...
61,CRIMINAL SIMULATION,1
62,FAIL TO PROVIDE FOR FUNCTIONALLY IMPAIRED PERSON,1
63,DISRUPTING PUBLIC SERVICE,1
64,INTIMIDATION-PUBLIC SERV/PARTY OFF,1


And with that we conclude the first post of the blog!  I hope you found this information helpful and I can't wait to deliver more content that explores the intersection of cities and data science.